In [1]:
import numpy as np
import cirq
from cirq import(
    X,
    Y,
    Z,
    CNOT,
    rx,
    rz,
    Circuit, 
    ops,
)
from mitiq import Executor, Observable, PauliString, pec
from mitiq.interface import mitiq_cirq
from mitiq.pec.representations import learn_biased_noise_parameters, represent_operation_with_local_biased_noise

In [2]:
a, b = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    rx(0.1).on(a),
    rx(-0.72).on(b),
    rz(0.4).on(a),
    rz(0.2).on(b),
    CNOT.on(a, b),
    rx(-0.1).on(b),
    rz(-0.23).on(a),
    CNOT.on(b, a),
    rx(-0.112).on(a),
    )

In [3]:
def simulator(circuit: cirq.Circuit) -> np.ndarray:
    return mitiq_cirq.compute_density_matrix(circuit, noise_level=(0.0,))
ideal_executor = Executor(simulator)

Apply biased noise (Mixed Unitary) channel after each sequence (want this to be equivalent to our test case). 
Then use biased noise representations to "perfectly" mitigate the errors.
Then vary the initial noise parameters slightly so some "learning" has to be performed. Calculate Choi, does overhead also make sense here?

In [4]:
# Define biased noise channel
from cirq import I, X, Y, Z, unitary
epsilon = 0.1
eta = 10
a = 1 - epsilon
b = epsilon * (3 * eta + 1) / (3 * (eta + 1))
c = epsilon / (3 * (eta + 1))

mix = [
    (a, unitary(I)),
    (b, unitary(Z)),
    (c, unitary(X)),
    (c, unitary(Y)),
    ]

    
def noisy_executor(circ: Circuit) -> np.ndarray:
    return mitiq_cirq.compute_density_matrix(circ, noise_model = ops.MixedUnitaryChannel(mix).on_each(a,b))

In [5]:
obs = Observable(PauliString("Z", support=(1,)))

In [6]:
epsilon, eta = learn_biased_noise_parameters(operation=rx, circuit=circuit, ideal_executor=ideal_executor,
       noisy_executor=noisy_executor, num_training_circuits=10, epsilon0=0.09, eta0=9.9, observable=obs)

TypeError: unsupported operand type(s) for *: 'int' and 'function'

In [ ]:
representations = [ ]
for operation in circuit:
    representations.append(represent_operation_with_local_biased_noise(operation, circuit, ))

In [ ]:
pec_value, pec_data = pec.execute_with_pec(
    circuit=circuit,
    observable=obs,
    executor=noisy_executor,
    representations=representations,
    num_samples = 1000,
    full_output=True,
    random_state = np.random.RandomState(7)
)